# 1. model 생성을 위한 units = 0이 아닌 데이터만 뽑아서 만든다

## 1-1 전체 데이터 load

In [1]:
from jw_package import *
from filling_missing import *
from functools import *

import configuration completed !
train + key + weather merging started
train + key + weather merging finished
function configuration completed ! 
Good to go !


In [2]:
each_station = []

for station_nbr in range(1,21):
    file_path = '../data/weather_partial_mean_all_best_close/{:02}.csv'.format(station_nbr)
    station = pd.read_csv(file_path,index_col=0)
    each_station.append(station)

weather = reduce((lambda x,y : x.append(y)),each_station)

In [3]:
sales = pd.read_csv('../data/basic/train.csv')
keys = pd.read_csv('../data/basic/key.csv')

In [4]:
data = sales.merge(keys).merge(weather)

In [5]:
data.head()

,date,store_nbr,item_nbr,units,station_nbr,avgspeed,codesum,cool,day,depart,...,sealevel,snowfall,stnpressure,sunrise,sunset,tavg,tmax,tmin,wetbulb,year
0,2012-01-01,1,1,0,1,4.6,RA FZFG BR,0.0,1,8.0,...,29.92,0.0,29.78,-,-,42.0,52.0,31.0,40.0,2012
1,2012-01-01,1,2,0,1,4.6,RA FZFG BR,0.0,1,8.0,...,29.92,0.0,29.78,-,-,42.0,52.0,31.0,40.0,2012
2,2012-01-01,1,3,0,1,4.6,RA FZFG BR,0.0,1,8.0,...,29.92,0.0,29.78,-,-,42.0,52.0,31.0,40.0,2012
3,2012-01-01,1,4,0,1,4.6,RA FZFG BR,0.0,1,8.0,...,29.92,0.0,29.78,-,-,42.0,52.0,31.0,40.0,2012
4,2012-01-01,1,5,0,1,4.6,RA FZFG BR,0.0,1,8.0,...,29.92,0.0,29.78,-,-,42.0,52.0,31.0,40.0,2012


## 1-2 tmp 데이터로 copy후 units = 0이 아닌 것만 뽑아서 model 생성

In [6]:
tmp = data.copy()

In [7]:
tmp = tmp[tmp['units']!=0]

In [8]:
etc = ['day','month','year','station_nbr']
category = ['sunrise','sunset','codesum']

for each in etc+category:
    tmp.drop(each,axis=1,inplace=True)

In [9]:
del tmp['tavg']
del tmp['sealevel']
del tmp['wetbulb']
del tmp['stnpressure']
del tmp['tmin']
del tmp['tmax']
del tmp['dewpoint']
del tmp['avgspeed']
del tmp['resultspeed']

In [10]:
tmp.reset_index(drop = True, inplace = True)

In [11]:
tmp.tail()

,date,store_nbr,item_nbr,units,cool,depart,heat,preciptotal,resultdir,snowfall
118691,2014-10-29,35,66,14,1.866667,4.8,4.466667,0.130333,23.0,0.0
118692,2014-10-30,35,16,16,1.866667,4.8,4.466667,0.130333,30.0,0.0
118693,2014-10-30,35,66,6,1.866667,4.8,4.466667,0.130333,30.0,0.0
118694,2014-10-31,35,16,18,1.866667,4.8,4.466667,0.130333,5.0,0.0
118695,2014-10-31,35,66,10,1.866667,4.8,4.466667,0.130333,5.0,0.0


# 2. 이부분은 codesum_v1.csv 데이터 load

In [15]:
codesum = pd.read_csv('../data/weather_v1.csv', index_col = 0)

In [16]:
codesum = codesum.merge(keys, on = 'station_nbr' )

In [17]:
codesum = codesum.loc[:,['date','store_nbr','is_holiday','codesum']]

In [18]:
tmp = tmp.merge(codesum, on = ['date','store_nbr'])

In [19]:
tmp.head()

,date,store_nbr,item_nbr,units,cool,depart,heat,preciptotal,resultdir,snowfall,is_holiday,codesum
0,2012-01-01,1,9,29,0.0,8.0,23.0,0.05,20.0,0.0,holiday,Not_MO
1,2012-01-01,1,28,2,0.0,8.0,23.0,0.05,20.0,0.0,holiday,Not_MO
2,2012-01-01,1,51,1,0.0,8.0,23.0,0.05,20.0,0.0,holiday,Not_MO
3,2012-01-02,1,9,60,0.0,11.0,24.0,0.01,24.0,0.0,holiday_work,MO
4,2012-01-02,1,28,5,0.0,11.0,24.0,0.01,24.0,0.0,holiday_work,MO


## 2-1. codesum에서 holiday 컬럼, codesum 컬럼 category변환 함수 
- holiday 이면 1 아니면 0
- codesum 'Not_MO'이면 1 MO 이면 0 

In [20]:
def cate_holiday(holiday):
    if holiday == 'holiday' or holiday == 'holiday_work':
            return 1
    else:
        return 0
def cate_codesum(codesum):
    if codesum == 'MO':
        return 0 
    else:
        return 1
    
tmp['is_holiday'] = tmp['is_holiday'].apply(cate_holiday)
tmp['codesum'] = tmp['codesum'].apply(cate_codesum)

In [27]:
tmp.head()

,date,store_nbr,item_nbr,units,cool,depart,heat,preciptotal,resultdir,snowfall,is_holiday,codesum
0,2012-01-01,1,9,29,0.0,8.0,23.0,0.05,20.0,0.0,1,1
1,2012-01-01,1,28,2,0.0,8.0,23.0,0.05,20.0,0.0,1,1
2,2012-01-01,1,51,1,0.0,8.0,23.0,0.05,20.0,0.0,1,1
3,2012-01-02,1,9,60,0.0,11.0,24.0,0.01,24.0,0.0,1,0
4,2012-01-02,1,28,5,0.0,11.0,24.0,0.01,24.0,0.0,1,0


# 3. model 생성
- formula = 'units ~ C(store_nbr) + C(item_nbr)+ scale(cool) + scale(heat) + scale(preciptotal) + scale(snowfall) + is_holiday'

In [28]:
formula = 'units ~ C(store_nbr) + C(item_nbr)+ scale(cool) + scale(heat) + scale(preciptotal) + scale(snowfall) + is_holiday'
model = sm.OLS.from_formula(formula, data= tmp)
result = model.fit()

In [29]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  units   R-squared:                       0.593
Model:                            OLS   Adj. R-squared:                  0.593
Method:                 Least Squares   F-statistic:                     1088.
Date:                Tue, 13 Mar 2018   Prob (F-statistic):               0.00
Time:                        00:12:33   Log-Likelihood:            -5.7588e+05
No. Observations:              118696   AIC:                         1.152e+06
Df Residuals:                  118536   BIC:                         1.154e+06
Df Model:                         159                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             15.0595      2

## 3-1. cross validation 확인

In [30]:
from sklearn.model_selection import KFold

cv = KFold(n_splits=4, shuffle=True, random_state=0)
for train_index, test_index in cv.split(X):
    print("test index :", test_index, len(test_index))
    print("." * 80)
    print("train index:", train_index, len(train_index))
    print("=" * 80)

    X_train = X.loc[train_index, :]
    X_test = X.loc[test_index, :]
    y_train = y.loc[train_index, :]
    y_test = y.loc[test_index, :]
    
    tmp = pd.concat([X_train, y_train], axis = 1)
    formula = 'units ~ C(store_nbr) + C(item_nbr)+ scale(cool) + scale(heat) + scale(preciptotal) + scale(snowfall) + is_holiday'
    
    model = sm.OLS.from_formula(formula, data= tmp)
    result = model.fit()
    
    
    y_hat = result.predict(X_test)
    
    
    
    print(((y_hat - y_hat.mean())**2).sum()  / ((y_test-y_test.mean())**2).sum() )

test index : [     3      6      9 ..., 118691 118694 118695] 29674
................................................................................
train index: [     0      1      2 ..., 118690 118692 118693] 89022
units    0.6848
dtype: float64
test index : [     1      8     11 ..., 118684 118688 118689] 29674
................................................................................
train index: [     0      2      3 ..., 118693 118694 118695] 89022
units    0.414393
dtype: float64
test index : [     0      4      5 ..., 118686 118690 118692] 29674
................................................................................
train index: [     1      2      3 ..., 118693 118694 118695] 89022
units    0.704704
dtype: float64
test index : [     2     10     13 ..., 118671 118681 118693] 29674
................................................................................
train index: [     0      1      3 ..., 118692 118694 118695] 89022
units    0.696493
dtype: float64


# 4. Outlier 제거 코드

In [33]:
res = result.resid_pearson
index = result.resid.index
outlier_index = abs(res) >= 2

row = pd.Series(index=index, data=res)
drop_index = row[outlier_index].index
len(drop_index)

tmp.drop(list(drop_index),inplace=True)
tmp.reset_index(drop = True, inplace = True)

X = tmp.loc[:, ['date', 'store_nbr', 'item_nbr', 'cool', 'heat', 'preciptotal','snowfall', 'is_holiday']]
y = tmp.loc[:, ['units']]

## 4-1. 2차 제거 후 cross_val 확인
- 위 코드 두번 실행

In [34]:
from sklearn.model_selection import KFold

cv = KFold(n_splits=4, shuffle=True, random_state=0)
for train_index, test_index in cv.split(X):
    print("test index :", test_index, len(test_index))
    print("." * 80)
    print("train index:", train_index, len(train_index))
    print("=" * 80)

    X_train = X.loc[train_index, :]
    X_test = X.loc[test_index, :]
    y_train = y.loc[train_index, :]
    y_test = y.loc[test_index, :]
    
    tmp = pd.concat([X_train, y_train], axis = 1)
    formula = 'units ~ C(store_nbr) + C(item_nbr)+ scale(cool) + scale(heat) + scale(preciptotal) + scale(snowfall) + is_holiday'
    
    model = sm.OLS.from_formula(formula, data= tmp)
    result = model.fit()
    
    
    y_hat = result.predict(X_test)
    
    
    
    print(((y_hat - y_hat.mean())**2).sum()  / ((y_test-y_test.mean())**2).sum() )

test index : [    0     3     6 ..., 61033 61034 61044] 15262
................................................................................
train index: [    1     2     4 ..., 61042 61043 61045] 45784
units    0.810859
dtype: float64
test index : [    1     4     9 ..., 61042 61043 61045] 15262
................................................................................
train index: [    0     2     3 ..., 61038 61039 61044] 45784
units    0.825858
dtype: float64
test index : [    2     7     8 ..., 61030 61035 61038] 15261
................................................................................
train index: [    0     1     3 ..., 61043 61044 61045] 45785
units    0.832029
dtype: float64
test index : [    5    10    11 ..., 61024 61032 61039] 15261
................................................................................
train index: [    0     1     2 ..., 61043 61044 61045] 45785
units    0.801574
dtype: float64


# 4-2. 현재모델

In [35]:
formula = 'units ~ C(store_nbr) + C(item_nbr)+ scale(cool) + scale(heat) + scale(preciptotal) + scale(snowfall) + is_holiday'
model = sm.OLS.from_formula(formula, data= tmp)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  units   R-squared:                       0.818
Model:                            OLS   Adj. R-squared:                  0.817
Method:                 Least Squares   F-statistic:                     1289.
Date:                Tue, 13 Mar 2018   Prob (F-statistic):               0.00
Time:                        00:16:49   Log-Likelihood:            -1.8560e+05
No. Observations:               45785   AIC:                         3.715e+05
Df Residuals:                   45625   BIC:                         3.729e+05
Df Model:                         159                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              7.9151      1

# 5. test.csv load

In [36]:
test = pd.read_csv('../data/basic/test.csv', index_col = 0)

In [38]:
test.reset_index(inplace = True)

In [40]:
test.tail()

,date,store_nbr,item_nbr
526912,2014-10-26,45,107
526913,2014-10-26,45,108
526914,2014-10-26,45,109
526915,2014-10-26,45,110
526916,2014-10-26,45,111


## 5-1. test 데이터에 weather's feature merge후 test_df로 저장

In [41]:
test_df = test.merge(keys).merge(weather)

In [42]:
test_df.tail()

,date,store_nbr,item_nbr,station_nbr,avgspeed,codesum,cool,day,depart,dewpoint,...,sealevel,snowfall,stnpressure,sunrise,sunset,tavg,tmax,tmin,wetbulb,year
526912,2014-08-02,40,107,13,6.5,,1.0,2,7.0,47.0,...,30.12,0.0,24.44,-,-,66.0,81.0,51.0,56.0,2014
526913,2014-08-02,40,108,13,6.5,,1.0,2,7.0,47.0,...,30.12,0.0,24.44,-,-,66.0,81.0,51.0,56.0,2014
526914,2014-08-02,40,109,13,6.5,,1.0,2,7.0,47.0,...,30.12,0.0,24.44,-,-,66.0,81.0,51.0,56.0,2014
526915,2014-08-02,40,110,13,6.5,,1.0,2,7.0,47.0,...,30.12,0.0,24.44,-,-,66.0,81.0,51.0,56.0,2014
526916,2014-08-02,40,111,13,6.5,,1.0,2,7.0,47.0,...,30.12,0.0,24.44,-,-,66.0,81.0,51.0,56.0,2014


In [45]:
etc = ['day','month','year', 'tavg', 'sealevel', 'wetbulb', 'stnpressure', 'tmin', 'tmax', 'dewpoint', 'avgspeed', 'resultspeed']
category = ['sunrise','sunset','codesum']

for each in etc+category:
    test_df.drop(each,axis=1,inplace=True)

In [46]:
test_df.tail()

,date,store_nbr,item_nbr,station_nbr,cool,depart,heat,preciptotal,resultdir,snowfall
526912,2014-08-02,40,107,13,1.0,7.0,0.0,0.0,17.0,0.0
526913,2014-08-02,40,108,13,1.0,7.0,0.0,0.0,17.0,0.0
526914,2014-08-02,40,109,13,1.0,7.0,0.0,0.0,17.0,0.0
526915,2014-08-02,40,110,13,1.0,7.0,0.0,0.0,17.0,0.0
526916,2014-08-02,40,111,13,1.0,7.0,0.0,0.0,17.0,0.0


In [64]:
test_df = test_df.merge(codesum, on = ['date','store_nbr'])
test_df['is_holiday'] = test_df['is_holiday'].apply(cate_holiday)
test_df['codesum'] = test_df['codesum'].apply(cate_codesum)

## 5-2. test_df에서 팔린 units의 feature에 대해서만 선택후 test_X_df에 저장

In [65]:
test_X_df = pd.DataFrame(columns=['date', 'store_nbr', 'item_nbr', 'station_nbr', 'cool', 'depart', 'heat', 'preciptotal', 'resultdir', 'snowfall', 'is_holiday', 'codesum'])

for nbr in range(1, 45+1):
    store_nbr = tmp[tmp['store_nbr'].isin([nbr])]
    sold_item_list = list(store_nbr['item_nbr'].unique())
    sold_item_list

    test_store_nbr = test_df[test_df['store_nbr'].isin([nbr])]
    to_test_df = test_store_nbr[test_store_nbr['item_nbr'].isin(sold_item_list)]

    test_X_df = pd.concat([test_X_df, to_test_df], axis = 0)
    
test_X_df.shape

(26086, 12)

In [66]:
test_X_df.head()

,date,store_nbr,item_nbr,station_nbr,cool,depart,heat,preciptotal,resultdir,snowfall,is_holiday,codesum
484301,2013-06-04,1,9,1,0.0,5.0,3.0,0.0,31.0,0.0,0,0
484320,2013-06-04,1,28,1,0.0,5.0,3.0,0.0,31.0,0.0,0,0
484332,2013-06-04,1,40,1,0.0,5.0,3.0,0.0,31.0,0.0,0,0
484339,2013-06-04,1,47,1,0.0,5.0,3.0,0.0,31.0,0.0,0,0
484343,2013-06-04,1,51,1,0.0,5.0,3.0,0.0,31.0,0.0,0,0


In [67]:
test_X_df.head()

,date,store_nbr,item_nbr,station_nbr,cool,depart,heat,preciptotal,resultdir,snowfall,is_holiday,codesum
484301,2013-06-04,1,9,1,0.0,5.0,3.0,0.0,31.0,0.0,0,0
484320,2013-06-04,1,28,1,0.0,5.0,3.0,0.0,31.0,0.0,0,0
484332,2013-06-04,1,40,1,0.0,5.0,3.0,0.0,31.0,0.0,0,0
484339,2013-06-04,1,47,1,0.0,5.0,3.0,0.0,31.0,0.0,0,0
484343,2013-06-04,1,51,1,0.0,5.0,3.0,0.0,31.0,0.0,0,0


## 5-3. test_df와 type을 맞춰줘야함 그래야 predict()에 넣을 수 있다.

In [68]:
test_X_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26086 entries, 484301 to 514090
Data columns (total 12 columns):
date           26086 non-null object
store_nbr      26086 non-null object
item_nbr       26086 non-null object
station_nbr    26086 non-null object
cool           26086 non-null float64
depart         26086 non-null float64
heat           26086 non-null float64
preciptotal    26086 non-null float64
resultdir      26086 non-null float64
snowfall       26086 non-null float64
is_holiday     26086 non-null object
codesum        26086 non-null object
dtypes: float64(6), object(6)
memory usage: 2.6+ MB


In [69]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 526917 entries, 0 to 526916
Data columns (total 12 columns):
date           526917 non-null object
store_nbr      526917 non-null int64
item_nbr       526917 non-null int64
station_nbr    526917 non-null int64
cool           526917 non-null float64
depart         526917 non-null float64
heat           526917 non-null float64
preciptotal    526917 non-null float64
resultdir      526917 non-null float64
snowfall       526917 non-null float64
is_holiday     526917 non-null int64
codesum        526917 non-null int64
dtypes: float64(6), int64(5), object(1)
memory usage: 52.3+ MB


In [70]:
test_X_df['store_nbr'] = test_X_df['store_nbr'].astype(int)
test_X_df['item_nbr'] = test_X_df['item_nbr'].astype(int)
test_X_df['station_nbr'] = test_X_df['station_nbr'].astype(int)
test_X_df['is_holiday'] = test_X_df['is_holiday'].astype(int)
test_X_df['codesum'] = test_X_df['codesum'].astype(int)

## 5-4. 이제 test_X_df 를 predict() 하면

In [71]:
y_hat = result.predict(test_X_df)

In [72]:
y_df = test_df.loc[:,['date','store_nbr','item_nbr']]

In [73]:
y_df['units'] = y_hat

In [76]:
y_df.tail()

,date,store_nbr,item_nbr,units
526912,2014-08-02,40,107,NaN
526913,2014-08-02,40,108,NaN
526914,2014-08-02,40,109,NaN
526915,2014-08-02,40,110,NaN
526916,2014-08-02,40,111,NaN


In [78]:
def itiszero(units):
    if units > 0:
        return units
    else:
        return 0

In [80]:
y_df['units'] = y_df['units'].apply(itiszero)

In [81]:
y_df['units'] = y_df['units'].astype(int)

In [83]:
y_df

,date,store_nbr,item_nbr,units
0,2013-04-01,2,1,0
1,2013-04-01,2,2,0
2,2013-04-01,2,3,0
3,2013-04-01,2,4,0
4,2013-04-01,2,5,58
5,2013-04-01,2,6,0
6,2013-04-01,2,7,0
7,2013-04-01,2,8,0
8,2013-04-01,2,9,0
9,2013-04-01,2,10,0


# 6. 제출 형식에 맞게 수정

In [84]:
y_df['id'] = y_df['store_nbr'].astype(str)+"_"+y_df['item_nbr'].astype(str)+'_'+y_df['date'].astype(str)

In [85]:
for_submission = y_df.loc[:,['id','units']]

In [87]:
for_submission.head()

,id,units
0,2_1_2013-04-01,0
1,2_2_2013-04-01,0
2,2_3_2013-04-01,0
3,2_4_2013-04-01,0
4,2_5_2013-04-01,58
